In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer
from sklearn.dummy import DummyClassifier

In [3]:
#manage date into form of yyyy/mm
def date(data):
    series=list(data)
    for num in range(len(series)):
        temp=series[num].split('/')[:2]
        series[num]=str(temp[0])+'/'+temp[1]
    return pd.Series(series)

In [4]:
def new_column(data,number):
    data[data>=number]=True
    data[data<number]=False
    return data

In [25]:
#read the file
data=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
shot_id=test.shot_id

#delete the row of lat, lon, and shot_id
del data["shot_id"],data["lat"],data["lon"]
del test["shot_id"],test["lat"],test["lon"]

#delete data with only showing up once
type_list=list(pd.value_counts(data.action_type).keys())
del_list=type_list[:-7:-1]  #there are 7 types with only one row data
del_index=[]
for k in range(len(del_list)):
    series=data.action_type
    del_index.extend(series[series==del_list[k]].index.tolist()) #get the certain type index
    
for i in range(len(del_list)):
    data=data.drop(del_index[i],axis=0) #delete rows by index
    
#create new columns "behind"(if behind the basket) and "side"(if the player on this side)
data["behind"],test["behind"]=pd.Series([0]*len(data)),pd.Series([0]*len(test))
data["behind"]=new_column(data["behind"],20)
test["behind"]=new_column(test["behind"],20)

data["side"],test["side"]=pd.Series([0]*len(data)),pd.Series([0]*len(test))
data["side"]=new_column(data["side"],20)
test["side"]=new_column(test["side"],20)

index=list(data.keys())  #index with all columns  

#the list of mananging column of "season"
arr=['1999-00','2012-13','2015-16','1997-98','1998-99','2014-15','1996-97','2013-14']

#preprocess each data column
for num in range(len(index)):
    if index[num]=="loc_x":
        data.loc_x=abs(abs(data.loc_x)-125)
        test.loc_x=abs(abs(test.loc_x)-125)
        
    elif index[num]=="loc_y":
        data.loc_y[data.loc_y<0]=abs(data.loc_y)
        test.loc_y[test.loc_y<0]=abs(test.loc_y)
        
    elif index[num]=="period":
        series,series1=data[index[num]],test[index[num]]
        series[series>4]=5
        series1[series1>4]=5
        
    elif index[num]=="seconds_remaining":
        data.seconds_remaining=(data.seconds_remaining+data.minutes_remaining*60)/10
        test.seconds_remaining=(test.seconds_remaining+test.minutes_remaining*60)/10
                 
    elif index[num]=="game_date":
        data["game_date"]=date(data["game_date"])
        test["game_date"]=date(test["game_date"])
    
    elif index[num]=="season":
        for a in range(len(arr)):
            ser1,ser2=data["season"],test["season"]
            ser1[ser1==arr[a]]="12-Jan"
            ser2[ser2==arr[a]]="12-Jan"
    
#turn the certain columns into specific values by using pd.get_dummies() function
action_type=data.action_type
label=data.action_type.values
expect=["shot_distance","loc_y","seconds_remaining","shot_distance","loc_y"]  #unnecessary to convert
df=data[expect]
title=["shot_zone_area","shot_made_flag","behind","season",'playoffs',"opponent",'period',"game_date"] 
feature=data[title]
feature_dummies=pd.get_dummies(feature,columns=title)
data=pd.concat([df,feature_dummies],axis=1)  #merge the converted columns and the other columns
#convert values to test data
feature2=test[title]
dummies=pd.get_dummies(feature2,columns=title)
df2=test[expect]
test=pd.concat([df2,dummies],axis=1)

#set up of train and test data
train_data,test_data,train_label,test_label=train_test_split(data,action_type,random_state=0,test_size=0.2,stratify=action_type)
imp=Imputer()  #impute the missing values
imp.fit(train_data)
train_1=imp.transform(train_data)
test_1=imp.transform(test_data)
test2=imp.transform(test)

#predict by using RandomForest Algorithm
rf=RandomForestClassifier(n_estimators=80,random_state=10).fit(train_1,train_label)
print("Prediction accuracy: %f" % rf.score(test_1, test_label))
pred1=rf.predict(test_1)
pred=rf.predict(test2)

c:\users\1234\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
c:\users\1234\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\1234\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#in

Prediction accuracy: 0.708002


In [21]:
with open("ans_0414.csv",'w') as file:
    for num in range(len(pred)):
        line=str(shot_id[num])+','+pred[num]+'\n'
        file.write(line)